In [1]:
from saxobank.model import streaming
from saxobank.model.chart import charts
from saxobank.model import common
from aiohttp import client
import json
from saxobank.converter import as_request, from_response
from datetime import datetime
from typing import List

a = charts.ChartSample

now = datetime.now()
mdl = charts.GetResp(Data=[charts.ChartSample(
    CloseAsk=100.0,
    CloseBid=200.0,
    Time='2015-02-02T01:02:03Z')],
    DataVersion=1)
# mdl = charts.GetResp(DataVersion=1.3)

d = as_request(mdl)
print(d)
m = from_response(d, charts.GetResp)
print(m)
# print(json.dumps(as_request([mdl])))


/home/chromeuser/git/python-saxobank/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


check: Data[0] is typing.Union[saxobank.model.chart.charts.ChartSample, dict[str, typing.Any]]
check: DataVersion is <class 'int'>
check: ChartInfo is typing.Optional[typing.Dict[str, typing.Any]]
{'Data': [{'CloseAsk': 100.0, 'CloseBid': 200.0, 'Time': '2015-02-02T01:02:03+00:00'}], 'DataVersion': 1}
check: Data[0] is typing.Union[saxobank.model.chart.charts.ChartSample, dict[str, typing.Any]]


TypeError: issubclass() argument 2 cannot be a parameterized generic

In [3]:
from saxobank.model.chart import charts
from saxobank.converter import as_request
from dataclasses import fields
from typing import get_type_hints
from inspect import get_annotations
from json import loads

exp = loads('{"AssetType": "CfdOnIndex", "Uic": 1924.3, "Horizon": 100, "Mode": "UpTo"}')
mdl = charts.GetReq(**exp)
mdl = charts.GetResp(Data=[charts.ChartSample(
    CloseAsk=100.0,
    CloseBid=200.0,
    Time='2015-02-02T01:02:03Z')],
    DataVersion=1)
    
print(as_request(mdl))

from inspect import get_annotations
from saxobank.model.common import AssetType
from types import GenericAlias
# schema = get_type_hints(charts.GetReq)
# t = schema['AssetType']


# schema = get_annotations(charts.GetReq, eval_str=True)
schema = get_annotations(charts.GetResp, eval_str=True)
print(schema)
# t = schema['Data']
# print(t.__args__)


for field in fields(charts.GetResp):
    print(field)
    
    # print(field.name)
    t = schema[field.name]
    v = getattr(mdl, field.name)
    print(isinstance(t, GenericAlias))
    print(dir(t))
    print(v)
    # print(t.__origin__)
    if isinstance(t, GenericAlias) and t.__origin__ == list:
        for i, e in enumerate(v):
            print(t.__args__[0].__name__)
            print(type(e).__name__)
            print(type(v[0]).__name__)
    elif isinstance(v, t):
        print('ok')
    else:
        print('ng')


{'Data': [{'CloseAsk': 100.0, 'CloseBid': 200.0, 'Time': '2015-02-02T01:02:03+00:00'}], 'DataVersion': 1}
{'Data': list[typing.Union[saxobank.model.chart.charts.ChartSample, dict[str, typing.Any]]], 'DataVersion': <class 'int'>, 'ChartInfo': typing.Optional[typing.Dict[str, typing.Any]]}
Field(name='Data',type='list[Union[ChartSample, dict[str, Any]]]',default=<dataclasses._MISSING_TYPE object at 0x7ef90a00e890>,default_factory=<dataclasses._MISSING_TYPE object at 0x7ef90a00e890>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)
True
['__add__', '__args__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mro_entries__', '__mul__', '__ne__', '__new

In [1]:
from types import GenericAlias, UnionType
from typing import Union, Any
from saxobank.model.chart import charts
from inspect import get_annotations
from saxobank.model.common import AssetType
from saxobank.model.base import SaxobankModel2

def checksubclass(cls, classinfo):
    print(classinfo)
    if getattr(cls, '__origin__', None) == Union:
        print(cls.__args__)
        return all([issubclass(e, classinfo) for e in cls.__args__])
    return issubclass(cls, classinfo)

schema = get_annotations(charts.GetResp, eval_str=True)
t = schema['Data'].__args__[0]
# print(t)
# print(type(t))
# test = GenericAlias(UnionType, (AssetType, str))
test = Union[dict[str, Any], SaxobankModel2]
# print(test)
# print(type(test))
# print(t == test)
print(checksubclass(t, test))


/home/chromeuser/git/python-saxobank/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


typing.Union[dict[str, typing.Any], saxobank.model.base.SaxobankModel2]
(<class 'saxobank.model.chart.charts.ChartSample'>, dict[str, typing.Any])


TypeError: issubclass() argument 2 cannot be a parameterized generic

In [ ]:
from mypy import  